In [12]:

class Empty(Exception):
    """Error attempting to access an element from an empty container"""
    pass

class LinkedQueue:
    """FIFO queue implementation using a singly linked list for storage."""
    
    # ------------------------- nested Node class --------------------------
    class _Node:
        """Lightweight, nonpublic class for storing a singly linked node."""
        __slots__ = '_element', '_next'

        def __init__(self, element, next): 
            self._element = element 
            self._next = next
    
    def __init__(self):
        """Create an empty queue."""
        self._head = None 
        self._tail = None 
        self._size = 0

    def __len__(self):
        """Return the number of elements in the queue."""
        return self._size 
    
    def is_empty(self):
        """Return True if the queue is empty."""
        return self._size == 0
    
    def first(self):
        """Return (but do not remove) the element at the front of the queue."""
        if self.is_empty():
            raise Empty('Queue is empty')
        return self._head._element

    def dequeue(self):
        """Remove and return the first element of the queue (i.e., FIFO)
        Raise Empty exception if the queue is empty.
        """
        if self.is_empty():
            raise Empty('Queue is empty')
        answer = self._head._element 
        self._head = self._head._next
        self._size -= 1
        if self.is_empty():                      # special case as queue is empty
            self._tail = None                    # removed head had been the tail
        return answer       

    def enqueue(self, e):
        """Add an element to the back of queue"""   
        newest = self._Node(e, None)
        if self.is_empty():
            self._head = newest 
        else:
            self._tail._next = newest
        self._tail = newest
        self._size += 1
    
    def traverse(self, result, node = None):
        if not node:
            node = self._head
        result.append(node._element)
        if node._next:
            self.traverse(result, node._next)
        return result

class Tree: 
    """Abstract base class representing a tree structure"""

    # ---------- nested Position class -------------- 
    class Position:
        """An abstraction representing the location of a single element."""

        def element(self):
            """Return the element stored at this Position."""
            raise NotImplementedError('must be implemented by subclass.')

        def __eq__(self, other):
            """Return True if other Position represents the same location."""
            raise NotImplementedError('must be implemented by subclass.')
        
        def __ne__(self, other):
            """Return True if other does not represent the same location."""
            return not(self == other)
    
    # ------------- abstract methods that concrete subclass must support -----------
    def root(self):
        '''Return Position representing the tree's root (or None if empty)'''
        raise NotImplementedError('must be implmented by subclass')

    def parent(self, p):
        """Return Position representing p's parent(or None if p is root)."""
        raise NotImplementedError('must be implmented by subclass')
    
    def num_children(self, p):
        """Return the number of children that Position p has."""
        raise NotImplementedError('must be implmented by subclass')
    
    def children(self, p):
        """Generate an iteration of Positions representing p's children."""
        raise NotImplementedError('must be implemented by subclass')
    
    def __len__(self):
        """Return the total number of elements in the tree."""
        raise NotImplementedError('must be implemented by subclass')

    # --------- concrete methods implemented in this class ----------------
    def is_root(self, p):
        """Return True if Position p represents the root of the tree."""
        return self.root() == p 
    
    def is_leaf(self, p):
        """Return True if Position p does not have any children"""
        return self.num_children(p) == 0
    
    def is_empty(self):
        """Return True if the tree is empty."""
        return len(self) == 0
    
    def __iter__(self):
        """Generate an iteration of the tree's elements."""
        for p in self.positions():
            yield p.element()
    
    # --------------- Preorder Traversal --------------
    def positions(self):
        """Generate an iteration of the tree's positions."""
        return self.preorder()          # return entire preorder iteration

    def _subtree_preorder(self, p): 
        """Generate a preorder iteration of positions in subtree rooted at p."""
        yield p                        # visit p before its subtrees
        for c in self.children(p):
            for other in self._subtree_preorder(c):
                yield other
    
    def preorder(self): # generator
        """Generate a preorder iteration of positions in subtree rooted at p."""
        if not self.is_empty():
            for p in self._subtree_preorder(self.root()):        # start recursion
                yield p

    # --------------- Postorder Traversal --------------
    def postorder(self):
        """Generate a postorder iteration of positions in the tree."""
        if not self.is_empty():
            for p in self._subtree_postorder(self.root()):
                yield p
    
    def _subtree_postorder(self, p):
        """Generate a postorder iteration of positions in subtree rooted at p."""
        for c in self.children(p):                        # for each child c
            for other in self._subtree_postorder(c):     # do postorder of c's subtree
                yield other                              # yielding each to our caller
        yield p
    
    # -------------- Breadth-First Traversal -------------
    def breadthfirst(self):
        """Generate a breadth-first iteration of the positions of the tree."""
        if not self.is_empty():
            fringe = LinkedQueue()            # known positions not yet yielded
            fringe.enqueue(self.root())       # starting with the root
            while not fringe.is_empty():
                p = fringe.dequeue()          # remove from font of the queue 
                yield p                       # report this position 
                for c in self.children(p):
                    fringe.enqueue(c)

class BinaryTree(Tree):
    """Abstract base class representing a binary tree structure."""
    # ------------------- additional abstract methods ------------------
    def left(self, p):
        """Return a Position representing p's left child.
        Return None if p does not have a left child.
        """
        raise NotImplementedError('must be implemented by subclass')
    
    def right(self, p):
        """Return a Position representing p's right child.
        Return None if p does not have a right child.
        """
        raise NotImplementedError('must be implemented by subclass')
    
    # ------------------ concrete methods implemented in this class ------------------
    def sibling(self, p):
        """Return a Position representing p's sibling (or None if no sibling)."""
        parent = self.parent(p)
        if parent is None:
            return None 
        else:
            if p == self.left(parent):
                return self.right(parent)
            else:
                return self.left(parent)
    
    def children(self, p):
        """Generate an iteration of Positions representing p's children."""
        if self.left(p) is not None:
            yield self.left(p)
        if self.right(p) is not None:
            yield self.right(p)

    # ------- Inorder Traversal ------------ 
    def inorder(self):
        """Generate an inorder iteration of positions in the tree."""
        if not self.is_empty():
            for p in self._subtree_inorder(self.root()):
                yield p 

    def _subtree_inorder(self, p):
        """Generate an inorder iteration of positions in subtree rooted at p."""
        if self.left(p) is not None:        # if left child exists, traverse its subtree 
            for other in self._subtree_inorder(self.left(p)):
                yield other 
        yield p                             # visit p between its subtrees 
        if self.right(p) is not None:       # if right child exists, traverse its subtree 
            for other in self._subtree_inorder(self.right(p)):
                yield other 

    # override inherited version of the tree's positions.
    def positions(self):
        """Generate an iteration of the tree's positions."""
        return self.inorder()


class LinkedBinaryTree(BinaryTree):
    """Linked representation of a binary tree structure."""

    class _Node:           # Lightweight, nonpublic class for storing a node 
        __slots__ = '_element', '_parent', '_left', '_right'
        
        def __init__(self, element, parent=None, left=None, right=None):
            self._element = element 
            self._parent = parent 
            self._left = left 
            self._right = right 
    
    class Position(BinaryTree.Position):
        """An abstraction representing the location of a single element."""
        
        def __init__(self, container, node):
            """Constructure should not be invoked by user."""
            self._container = container 
            self._node = node 
        
        def element(self):
            return self._node._element
        
        def __eq__(self, other):
            """Return True if other is a Position representing the same location."""
            return type(other) is type(self) and other._node is self._node
    
    def _validate(self, p):
        """Return associated node, if position is valid."""
        if not isinstance(p, self.Position):
            raise TypeError('p must be proper Position type')
        
        if p._container is not self:
            raise ValueError('p does not belong to this container')
        
        if p._node._parent is p._node: # convention for deprecated nodes
            raise ValueError('p is no longer valid')
        return p._node

    def _make_position(self, node):
        """Return Position instance for given node (or None if no node)."""
        return self.Position(self, node) if node is not None else None
    
    # --------------- binary tree constructure -----------------------
    def __init__(self):
        """Create an initially empty binary tree"""
        self._root = None 
        self._size = 0
    
    # --------------- public accessors --------------------------
    def __len__(self):
        return self._size 
    
    def root(self):
        """Return the root Position of the tree (or None if tree is empty)."""
        return self._make_position(self._root)
    
    def parent(self, p):
        """Return the Position of p's parent (or None if p is root)"""
        node = self._validate(p)
        return self._make_position(node._parent)
    
    def left(self, p):
        """Return the Position of p's left child (or None if no left child)."""
        node = self._validate(p)
        return self._make_position(node._left)
    
    def right(self, p):
        """Return the Position of p's right child (or None if no right child)."""
        node = self._validate(p)
        return self._make_position(node._right)

    def num_children(self, p):
        """Return the number of children of Position p."""
        node = self._validate(p)
        count = 0
        if node._left is not None:
            count += 1 
        if node._right is not None:
            count += 1 
        return count 

    # --------- nonpublic update methods -------------------------
    def _add_root(self, e):
        """Place element e at the root of an empty tree and return new Position.
        Raise ValueError if tree nonempty.
        """
        if self._root is not None: raise ValueError('Root exists')
        self._size = 1 
        self._root = self._Node(e)
        return self._make_position(self._root)
    
    def _add_left(self, p, e):
        node = self._validate(p)
        if node._left is not None: raise ValueError('Left child exists')
        self._size += 1
        node._left = self._Node(e, node)
        return self._make_position(node._left)
    
    def _add_right(self, p, e):
        node = self._validate(p)
        if node._right is not None: raise ValueError('Right child exists')
        self._size += 1
        node._right = self._Node(e, node)
        return self._make_position(node._right)
    
    def _replace(self, p, e):
        """Replace the element at position p with e, and return old element."""
        node = self._validate(p)
        old = node._element 
        node._element = e 
        return old
    
    def _delete(self, p):
        """Delete the node at Position p, and replace it with its child, if any. 
        Return the element that had been stored at Position p.
        Raise ValueError if POsition p is in valid or p has two hildren.
        """
        node = self._validate(p)
        if self.num_children(p) == 2: raise ValueError('p has two children.')
        child = node._left if node._left else node._right 
        if child is not None:
            child._parent = node._parent 
        
        if node is self._root:
            self._root = child
        else:
            parent = node._parent 
            if node is parent._left:
                parent._left = child 
            else: 
                parent._right = child
        
        self._size -= 1
        node._parent = node           # convention for deprecated node
        return node._element
    
    def _attach(self, p, t1, t2):
        """Attach trees t1 and t2 as left and right subtrees of external p."""
        node = self._validate(p)
        if not self.is_leaf(p): raise ValueError('position must be leaf')
        if not type(self) is type(t1) is type(t2): # all 3 trees must be same type
            raise TypeError('Tree types must match.')
        self._size += len(t1) + len(t2)
        if not t1.is_empty():           # attached t1 as left subtree of node 
            t1._root._parent = node 
            node._left = t1._root
            t1._root = None             # set t1 instance to empty
            t1._size = 0 
        if not t2.is_empty():           # attached t2 as right subtree of node 
            t2._root._parent = node 
            node._right = t2._root 
            t2._root = None             # set t2 instance to empty
            t2._size = 0

    def depth(self, p):
        if self.is_root(p):
            return 0 
        else:
            return 1 + self.depth(self.parent(p))
        
    def _height2(self, p): # O(n)
        if self.is_leaf(p):
            return 0 
        else:
            return 1 + max(self._height2(c) for c in self.children(p))
    # R-8.4 What is the running time of a call to T._height2(p) when called on a position p
    # distinct from the root of T -> O(n - c) (c means the number of nodes in depth)

    def left_child_sum(self):
        """Generate a postorder iteration of positions in the tree."""
        if not self.is_empty():
            for p in self._subtree_postorder(self.root()):
                yield 1
    
    def _subtree_postorder(self, p):
        """Generate a postorder iteration of positions in subtree rooted at p."""
        for c in self.children(p):                        # for each child c
            for other in self._subtree_postorder(c):     # do postorder of c's subtree
                if c == self.left(p):
                    yield 1                             # yielding each to our caller
        yield 1


from collections.abc import MutableMapping

# MapBase (Abstract Data Type) : Extending the MutableMapping abstract base class
class MapBase(MutableMapping):
    """Our own abstract base class that includes a nonpublic _Item class."""
    # ------------------ nested Item class -----------------------------------

    class _Item:
        """Lightweight composite to store key-value pairs as map items."""
        __slots__ = '_key', '_value'

        def __init__(self, k, v):
            self._key = k 
            self._value = v 

        def __eq__(self, other):
            return self._key == other._key 

        def __ne__(self, other):
            return not (self == other)
    
        def __lt__(self, other):
            return self._key < other._key


class TreeMap(LinkedBinaryTree, MapBase):
    """Sorted map implementation using a binary search tree."""

    # -------------------- override Position class ----------------------
    class Position(LinkedBinaryTree.Position):
        def key(self):
            """Return key of map's key-value pair."""
            return self.element()._key

        def value(self):
            """Return value of map's key-value pair""" 
            return self.element()._value 

    #------------------------------- nonpublic utilities -------------------------------
    def _subtree_search(self, p, k): # binary tree search
        """Return Position of p's subtree having key k, or last node searched."""
        if k == p.key():
            return p
        elif k < p.key():
            if self.left(p) is not None:
                return self._subtree_search(self.left(p), k)
        else:
            if self.right(p) is not None:
                return self._subtree_search(self.right(p), k)
        return p 

    def _subtree_first_poisition(self, p):
        """"Return Position of first item in subtree rooted at p."""
        walk = p 
        while self.left(walk) is not None:                # keep walking left 
            walk = self.left(walk)
        return walk
    
    def _subtree_last_position(self, p):
        """Return Position of last item in subtree rooted at p."""
        walk = p 
        while self.right(walk) is not None:
            walk = self.right(walk)
        return walk 
     
    def first(self):
        """Return the first Position in the tree (or None if empty)."""
        return self._subtree_first_poisition(self.root()) if self(len) > 0 else None 

    def last(self):
        """Return the last Position in the tree (or None if empty)."""
        return self._subtree_last_position(self.root()) if len(self) > 0 else None 
    
    def before(self, p):
        """Return the Position just before p in the natural order.
        Retur None if p is the first position
        """
        self._validate(p)
        if self.left(p):
            return self._subtree_last_position(self.left(p))
        else:
            # walk upward
            walk = p 
            above = self.parent(walk)
            while above is not None and walk == self.left(above):
                walk = above 
                above = self.parent(walk)
            return above 
   
    def after(self, p):
        self._validate(p)
        if self.right(p):
            return self._subtree_last_position(self.right(p))
        else:
            walk = p
            above = self.parent(walk)
            while above is not None and walk == self.right(above):
                walk = above 
                above = self.parent(walk)
            return above 

    def find_min(self):
        """Return (key, value) pair with minimum key (or None if empty)."""
        if self.is_empty():
            return None 
        else:
            p = self.first() 
            return (p.key(), p.value())
    
    def find_ge(self, k):
        """Return (key, value) pair with least key greater than or equal to k.
        Return None if there does not exist such a key.
        """
        if self.is_empty():
            return None 
        else:
            p = self.find_position(self._root)
            if p.key() < k:
                p = self.after(p)                 # p's key is too small 
            return (p.key(), p.value()) if p is not None else None  
    
    def find_position(self, k):
        """Return position with key k, or else neighbor (or None if empty)"""
        if self.is_empty():
            return None 
        else:
            p = self._subtree_search(self.root(), k)
            # self._rebalance_access(p)
            return p 

    def find_range(self, start, stop):
        """"
        Iterate all (key, value) pairs such taht start <= key < stop.

        If start is None, iteration begins with minimum key of map.
        If stop is None, iteration continues through the maximum key of map.
        """
        if not self.is_empty():
            if start is None:
                p = self.first()
            else:
                p = self.find_position(start)
                if p.key() < start:
                    p = self.after(p)
                while p is not None and (stop is None or p.key() < stop):
                    yield (p.key(), p.value())
                    p = self.after(p)
    
    def __getitem__(self, k):
        """Return value associated with key k (raise KeyError if not found.)"""
        if self.is_empty():
            raise KeyError('Key Error' + repr(k))
        else:
            p = self._subtree_search(self.root(), k)
            # self._rebalance_access(p)
            if k != p.key():
                raise KeyError('Key Error' + repr(k))
            return p.value()
    
    def __setitem__(self, k, v):
        """Assign value v to key k, overwriting existing value if present."""
        if self.is_empty():
            leaf = self._add_root(self._Item(k, v))
        else:
            p = self._subtree_search(self.root(), k)
            if p.key() == k:
                p.element()._value = v
                # self._rebalance_access(p)
                return 
            else:
                item = self._Item(k, v)
                if p.key() < k:
                    leaf = self._add_right(p, item)
                else:
                    leaf = self._add_left(p, item)
        # self._rebalance_insert(leaf)
    
    def __iter__(self):
        """Generate an iteration of all keys in the map in order."""
        p = self.first()
        while p is not None:
            yield p.key()
            p = self.after(p)                
    
    def delete(self, p):
        """Remove the item at given Position."""
        self._validate(p)
        if self.left(p) and self.right(p):
            replacement = self._subtree_last_position(self.left(p))
            self._replace(p, replacement.element())
            p = replacement
        # now p has at most one child
        parent = self.parent(p)
        self._delete(p)
        # self._rebalance_delete(parent)

    def __delitem__(self, k):
        if not self.is_empty():
            p = self._subtree_search(self.root(), k)
            if k == p.key():
                self.delete(p)
                return 
            # self._rebalance_access(p)
        raise KeyError('Key Error' + repr(k))
    
    def _rebalance_insert(self, p): pass 
    def _rebalance_delete(self, p): pass 
    def _rebalance_access(self, p): pass 

    def _relink(self, parent, child, make_left_child):
        """Relink parent node with child node (we allow child to be None.)"""
        if make_left_child:
            parent._left = child                   # make it a left child 
        else:
            parent._right = child                  # make it a right child 
        if child is not None:
            child._parent = parent

    def _rotatte(self, p):
        """Rotate Position p above its parent."""
        x = p._node 
        y = x._parent 
        z = y._parent 
        if z is None:
            self._root = x
            x._parent = None 
        else:
            self._relink(z, x, y == z._left) 
        # now rotate x and y, including transfer of middle subtree
        if x == y._left:
            self._relink(y, x._right, True)        # x._right becomes left child of y
            self._relink(x, y, False)              # y becomes right child of x
        else:
            self._relink(y, x._left, False)        # x._left becomes right child of y
            self._relink(x, y, True)               # y becomes left child of x

        def _restructure(self, x):
            """Perform trinode restructure of Position x with parent/grandparent."""
            y = self.parent(x)
            z = self.parent(y)
            if (x == self.right(y)) == (y == self.right(z)):
                self._rotate(y)                      # single rotation (of y)
                return y
            else:
                self._rotate(x)                      # double rotation (of x)
                self._rotate(x)
                return x 




TypeError: p must be proper Position type